In [11]:
# 주요 라이브러리 import
import warnings
warnings.filterwarnings(action='ignore')
import time
from IPython.display import Image
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import Ridge, Lasso, ElasticNet

# 디렉토리 및 파일 이름에 맞추어 변경
font_location = 'C:/Windows/Fonts/malgun.ttf'

font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)

In [12]:
df=pd.read_csv('2018 국민여행조사_국내여행.csv')

In [13]:
df

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
0,11010560291_702520,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11010560291_702521,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11010560291_702526,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11010560291_702539,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11010560291_702558,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24149,3902061054A_748235,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24150,3902061054A_748239,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24151,3902061054A_890309,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24152,3902061054A_890311,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df['D_TRA1_SYEAR'].value_counts()

2018.0    14296
2017.0        7
Name: D_TRA1_SYEAR, dtype: int64

In [20]:
df['D_TRA1_SDAY'].isna().sum()

9851

In [21]:
df['D_TRA1_COST'].isna().sum()

9851

In [29]:
df['A1'].isnull().sum()

12306

In [30]:
df['A1'].notnull().sum()

11848

In [32]:
## 추천도, 재방문의사 보통이상만 추출
df = df[(df['A12']>=3) & (df['A13']>=3)]
df

,ID,MON_EXP_DOM_TOUR,MON_EXP_DOM_BUSIN,MON_EXP_DOM_FRI,YEAR_EXP_DOM_TOUR,YEAR_EXP_DOM_BUSIN,YEAR_EXP_DOM_FRI,D_TRA1_CASE,D_TRA1_SYEAR,D_TRA1_SMONTH,...,국내_여행방문지_기타당일7,국내_여행방문지_기타당일8,국내_여행방문지_기타당일9,국내_여행방문지_기타당일10,국내_여행방문지_기타당일11,국내_여행방문지_기타당일12,국내_여행방문지_기타당일13,국내_여행방문지_기타당일14,국내_여행방문지_기타당일15,국내_여행방문지_기타당일16
3,11010560291_702539,1,2,2,NaN,NaN,NaN,2.0,2018.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,11010560321_500005,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,11010560321_500010,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11010560321_500029,1,2,2,NaN,NaN,NaN,2.0,2018.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11010600061_900004,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24115,3902057161A_748148,1,2,2,NaN,NaN,NaN,2.0,2018.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24119,39020590511_748173,1,2,1,NaN,NaN,NaN,2.0,2018.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24126,39020600441_1223043,1,2,2,1.0,2.0,1.0,2.0,2018.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24132,39020610101_922792,1,2,2,NaN,NaN,NaN,2.0,2018.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
